# 1. Data checking

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import decimal
import math
from datetime import datetime as dt
from datetime import timedelta
from tqdm import tqdm

## 1) yellow trip data from 2019 Jan

In [ ]:
y1901 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-01.parquet")
y1901.head(10)

In [ ]:
print(len(y1901))

In [ ]:
print(y1901.columns)
##let's do it columns by columns

In [ ]:
len(y1901.columns)

### -1. VendorID

In [ ]:
print(y1901.VendorID.unique())

What is 4,5...? But it doesn't matter

### -2. pickup & dropoff datetime

In [ ]:
y1901[y1901.tpep_pickup_datetime.dt.hour==1]

In [ ]:
y1901.tpep_pickup_datetime.dt.year.unique()

In [ ]:
y1901[(y1901.tpep_pickup_datetime.dt.year==2018)|
      (y1901.tpep_pickup_datetime.dt.year==2003)|
     (y1901.tpep_pickup_datetime.dt.year==2008)|
     (y1901.tpep_pickup_datetime.dt.year==2009)|
     (y1901.tpep_pickup_datetime.dt.year==2001)|
     (y1901.tpep_pickup_datetime.dt.year==2088)]

In [ ]:
y1901 = y1901[y1901.tpep_pickup_datetime.dt.year==2019]

In [ ]:
y1901.tpep_pickup_datetime.dt.month.unique()
##It is data for January!!

In [ ]:
y1901[y1901.tpep_pickup_datetime.dt.month!=1]

In [ ]:
y1901 = y1901[y1901.tpep_pickup_datetime.dt.month==1]
len(y1901)

In [ ]:
y1901.tpep_dropoff_datetime.dt.year.unique()

In [ ]:
y1901[y1901.tpep_dropoff_datetime.dt.year==2018]

In [ ]:
y1901 = y1901[y1901.tpep_dropoff_datetime.dt.year==2019]
len(y1901)

In [ ]:
y1901.tpep_dropoff_datetime.dt.month.unique()

In [ ]:
y1901[y1901.tpep_dropoff_datetime.dt.month==2]
##It can be because someone pickup January 31st.

In [ ]:
y1901=y1901[y1901.tpep_dropoff_datetime>y1901.tpep_pickup_datetime]

### -3. passenger_count

In [ ]:
y1901.passenger_count.unique()

In [ ]:
y1901[y1901.passenger_count.isna()]

In [ ]:
y1901[(y1901.passenger_count.isna()) | (y1901.passenger_count==0)]

In [ ]:
y1901 = y1901[(y1901.passenger_count.notnull()) & (y1901.passenger_count!=0)]
len(y1901)

### -4. trip_distance

In [ ]:
y1901.trip_distance.plot()
plt.show()

In [ ]:
y1901.trip_distance.max(),y1901.trip_distance.min()

In [ ]:
y1901[y1901.trip_distance==831.8]
##this should be erased by speed columns

In [ ]:
y1901[y1901.trip_distance==0.0]

In [ ]:
y1901 = y1901[y1901.trip_distance!=0.0]
print(len(y1901))

In [ ]:
y1901['trip_distance'] = y1901['trip_distance']*1.61
y1901.head()

### -5. RatecodeID

In [ ]:
y1901.RatecodeID.unique()

In [ ]:
y1901[y1901.RatecodeID==99.0]

### -6. Payment_type

In [ ]:
y1901.payment_type.unique()

### -7. Fare_amount

In [ ]:
y1901.fare_amount.plot()
plt.show()

In [ ]:
y1901[y1901.fare_amount>600000]

In [ ]:
y1901 = y1901[y1901.fare_amount<600000]

In [ ]:
y1901.fare_amount.plot()

In [ ]:
y1901[y1901.fare_amount>2000]

In [ ]:
y1901[y1901.fare_amount<0]

In [ ]:
y1901 = y1901[y1901.fare_amount>=0]

### -8. tip_amount
What I want to have

In [ ]:
y1901.tip_amount.plot()

In [ ]:
y1901[y1901.tip_amount==0]

In [ ]:
y1901[y1901.tip_amount!=0]

### -9. total_amount
is it really 'total'?

In [ ]:
y1901[y1901.fare_amount + y1901.extra + y1901.mta_tax + y1901.tip_amount + y1901.tolls_amount + y1901.improvement_surcharge!=y1901.total_amount]

In [ ]:
not_total_list = y1901[y1901.fare_amount + y1901.extra + y1901.mta_tax + y1901.tip_amount + y1901.tolls_amount + y1901.improvement_surcharge != y1901.total_amount].index
not_total_list[:10]

In [ ]:
not_total_list_index=[]
for i in tqdm(not_total_list):
    if not math.isclose(y1901.fare_amount[i]+y1901.extra[i]+y1901.mta_tax[i]+y1901.tip_amount[i]+y1901.tolls_amount[i]+y1901.improvement_surcharge[i],y1901.total_amount[i]):
        not_total_list_index.append(i)
print(len(not_total_list_index))

In [ ]:
y1901.drop(not_total_list_index,axis=0,inplace=True)
print(len(y1901))

In [ ]:
y1901.to_csv("D:/Working_Place/new_uber/data/data_processed/y1901.csv")

### -10. Congestion_surcharge & airport_fee

In [ ]:
y1901[y1901.congestion_surcharge.notnull()]

In [ ]:
y1901[(y1901.congestion_surcharge.notnull())&(y1901.congestion_surcharge!=0)]

In [ ]:
y1901['congestion_surcharge'] = y1901['congestion_surcharge'].fillna(0)

In [ ]:
y1901['airport_fee'] = y1901['airport_fee'].fillna(0)

### -11. Average Speed

In [ ]:
y1901.loc[68267,'tpep_dropoff_datetime']-y1901.loc[68267,'tpep_pickup_datetime']>timedelta(1)

In [ ]:
y1901[(y1901.tpep_dropoff_datetime-y1901.tpep_pickup_datetime)>timedelta(hours=12)]

In [ ]:
interval_time = y1901.loc[68267,'tpep_dropoff_datetime']-y1901.loc[68267,'tpep_pickup_datetime']
interval_time.total_seconds()/60/60

In [ ]:
y1901['average_speed'] = y1901.apply(lambda x: x['trip_distance']/((x['tpep_dropoff_datetime']-x['tpep_pickup_datetime']).total_seconds()/60/60),axis=1)
y1901

In [ ]:
y1901['average_speed'].plot()
plt.show()

In [ ]:
y1901[y1901.average_speed>20000]

In [ ]:
y1901['average_speed'].describe()

In [ ]:
q1=y1901['average_speed'].quantile(0.25)
q3=y1901['average_speed'].quantile(0.75)
iqr = q3-q1
len(y1901[(y1901.average_speed<(q3+1.5*iqr)) & (y1901.average_speed>(q1-1.5*iqr))])/len(y1901)

In [ ]:
y1901 = y1901[(y1901.average_speed<(q3+1.5*iqr)) & (y1901.average_speed>(q1-1.5*iqr))]
y1901.head()

In [ ]:
y1901.trip_distance.describe()

In [ ]:
y1901[y1901.trip_distance==3.445561e+02]

In [ ]:
y1901.to_csv('D:/Working_Place/new_uber/data/data_processed/y1901.csv')

In [ ]:
y1901.columns

# 2) Do it with all monthly data!

In [2]:
def dataprocessing(df,year,month):
    print("I'm doing dataprocessing with {}-{} data".format(year,month))
    print("Original length : ", len(df))
    ##1) column checking
    if not (df.columns==['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
           'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
           'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
           'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
           'total_amount', 'congestion_surcharge', 'airport_fee']).all():
            print("The columns are different!")
            print(df.columns)
            return df
    else:
        ##2) pickup/dropoff datetime
        df1=df[(df.tpep_pickup_datetime.dt.year==year)&(df.tpep_pickup_datetime.dt.month==month)]
        if month!=12:
            df1=df1[df1.tpep_dropoff_datetime.dt.year==year]
        df1=df1[df1.tpep_dropoff_datetime>df1.tpep_pickup_datetime]
        print("After datetime processing : ", len(df1))
            
        ##3) passenger_count
        df1 = df1[(df1.passenger_count.notnull()) & (df1.passenger_count>0)]
        print("After passenger_count processing : ", len(df1))

        ##4) trip_ditance
        df1 = df1[df1.trip_distance>0.0]
        print("After trip_distance processing : ", len(df1))

        ##5) fare_amount
        df1 = df1[df1.fare_amount>0]
        print("After fare_amount processing : ", len(df1))

        ##6) total_amount
        df1.loc[:,'total_amount'] = df1.loc[:,'fare_amount':'improvement_surcharge'].sum(axis=1)
        print("After total_amount processing : ", len(df1))
        
        ##7) congestion_surcharge
        df1['congestion_surcharge'] = df1['congestion_surcharge'].fillna(0)
        df1['airport_fee'] = df1['airport_fee'].fillna(0)
        print("After congestion_surcharge and airport_fee processing : ", len(df1))
        
        ##8) average_speed
        df1['average_speed'] = df1.apply(lambda x: x['trip_distance']/((x['tpep_dropoff_datetime']-x['tpep_pickup_datetime']).total_seconds()/60/60),axis=1)
        q1=df1['average_speed'].quantile(0.25)
        q3=df1['average_speed'].quantile(0.75)
        iqr = q3-q1
        df1 = df1[(df1.average_speed<(q3+1.5*iqr)) & (df1.average_speed>(q1-1.5*iqr))]
        print("After average_speed processing : ", len(df1))
        
        yearname = str(year%2000)
        if month<10:
            monthname = "0"+str(month)
        else:
            monthname = str(month)
        print("Saving to data_processed!")
        df1.to_parquet('D:/Working_Place/new_uber/data/data_processed/y{}{}.parquet'.format(yearname,monthname))
        return df1

## -1. test with 2019-02

In [ ]:
y1902 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-02.parquet")
print(len(y1902))
y1902.head()


In [ ]:
y1902_processed = dataprocessing(y1902,2019,2)
y1902_processed.head()

In [ ]:
y1902.to_csv('D:/Working_Place/new_uber/data/data_processed/y1902.csv')

## -2. With all data

In [ ]:
y1903 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-03.parquet")
y1904 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-04.parquet")
y1905 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-05.parquet")
y1906 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-06.parquet")
data_list = [(y1903,2019,3),(y1904,2019,4),(y1905,2019,5),(y1906,2019,6)]

In [ ]:
after_processing=[y1901,y1902]
for i,j,k in tqdm(data_list):
    after_processing.append(dataprocessing(i,j,k))

In [ ]:
y1907 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-07.parquet")
y1908 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-08.parquet")
y1909 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-09.parquet")
y1910 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-10.parquet")
y1911 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-11.parquet")
y1912 = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_2019-12.parquet")

data_list = [(y1907,2019,7),(y1908,2019,8),(y1909,2019,9),(y1910,2019,10),(y1911,2019,11),(y1912,2019,12)]

In [ ]:
for i,j,k in tqdm(data_list):
    after_processing.append(dataprocessing(i,j,k))

## -3. Do it first of all

In [3]:
for i in [2019,2020,2021,2022]:
    for j in tqdm(range(1,13)):
        if j>=10:
            dataframe_yellow_cab = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_{}-{}.parquet".format(i,j))
        else:
            dataframe_yellow_cab = pd.read_parquet("D:/Working_Place/new_uber/data/yellow_tripdata_{}-{}.parquet".format(i,"0"+str(j)))
        dataprocessing(dataframe_yellow_cab,i,j)
        del dataframe_yellow_cab 

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

I'm doing dataprocessing with 2019-1 data
Original length :  7696617
After datetime processing :  7689526
After passenger_count processing :  7544006
After trip_distance processing :  7496719
After fare_amount processing :  7490191
After total_amount processing :  7490191
After congestion_surcharge and airport_fee processing :  7490191
After average_speed processing :  7008294
Saving to data_processed!


  8%|██████▊                                                                           | 1/12 [03:35<39:29, 215.42s/it]

I'm doing dataprocessing with 2019-2 data
Original length :  7049370
After datetime processing :  7042349
After passenger_count processing :  6894016
After trip_distance processing :  6851861
After fare_amount processing :  6842999
After total_amount processing :  6842999
After congestion_surcharge and airport_fee processing :  6842999
After average_speed processing :  6424718
Saving to data_processed!


 17%|█████████████▋                                                                    | 2/12 [06:35<32:23, 194.34s/it]

...

 83%|████████████████████████████████████████████████████████████████████▎             | 10/12 [13:17<02:43, 81.79s/it]

I'm doing dataprocessing with 2022-11 data
Original length :  3252717
After datetime processing :  3249670
After passenger_count processing :  3081423
After trip_distance processing :  3040603
After fare_amount processing :  3018824
After total_amount processing :  3018824
After congestion_surcharge and airport_fee processing :  3018824
After average_speed processing :  2809211
Saving to data_processed!


 92%|███████████████████████████████████████████████████████████████████████████▏      | 11/12 [14:37<01:21, 81.13s/it]

I'm doing dataprocessing with 2022-12 data
Original length :  3399549
After datetime processing :  3397836
After passenger_count processing :  3211436
After trip_distance processing :  3165018
After fare_amount processing :  3140924
After total_amount processing :  3140924
After congestion_surcharge and airport_fee processing :  3140924
After average_speed processing :  2933756
Saving to data_processed!


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [15:58<00:00, 79.89s/it]
